In [1]:
import cv2
import tkinter as tk

# Function to get screen resolution
def get_screen_resolution():
    root = tk.Tk()
    width = root.winfo_screenwidth()
    height = root.winfo_screenheight()
    root.destroy()
    return width, height

# Load pre-trained HOG detector for pedestrian detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open video file
video_path = '6387-191695740_small.mp4'
cap = cv2.VideoCapture(video_path)

# Get screen resolution
screen_width, screen_height = get_screen_resolution()

# Calculate dimensions for half of the screen
target_width = screen_width // 2
target_height = screen_height // 2

# Initialize dictionary to track detected individuals
detected_individuals = {}

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to half of the screen dimensions
    frame = cv2.resize(frame, (target_width, target_height))

    # Convert frame to grayscale for pedestrian detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect pedestrians using HOG
    pedestrians, _ = hog.detectMultiScale(gray, winStride=(8, 8), padding=(16, 16), scale=1.05)

    # Update dictionary with detected individuals
    for (x, y, w, h) in pedestrians:
        centroid = (x + w // 2, y + h // 2)
        detected_individuals[centroid] = (x, y, w, h)

        # Draw green bounding box around the detected person
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display count of detected individuals near the green box
        cv2.putText(frame, f'People', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display count of detected individuals
    num_people = len(detected_individuals)
    cv2.putText(frame, f'People Count: {num_people}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    
    # Display the frame
    cv2.imshow('People Counting', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture object and close windows
cap.release()
cv2.destroyAllWindows()
